In [ ]:
#==========================================================================
# SCRIPT DE PROCESSAMENTO DE DADOS DE CORRENTE
#--------------------------------------------------------------------------
# Autor: Zydanne Costa
# Data: 02 de setembro de 2024
# Análise de Dados de Correntes - Correntometro JFE Advantech
# Descrição: Este script é destinado ao tratamento e visualização dos dados
# adquiridos de um correntômetro eletromagnético JFE Advantech. Os dados
# incluem informações sobre direção e velocidade das correntes. O script
# realiza a importação, manipulação e plotagem dos dados de forma detalhada
# e visualmente clara.
#--------------------------------------------------------------------------
# Última Modificação: 02 de setembro de 2024
#==========================================================================

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
import matplotlib.dates as mdates

# Carregando os dados
# Definindo o caminho da pasta com os dados brutos
path = '/SEU/CAMINHO/AQUI/NomeDoArquivo.csv'

# Carregar dados ignorando as 75 primeiras linhas de cabeçalho
data = pd.read_csv(path, skiprows=75)

# Manter apenas as colunas de 1 a 3 (data+hora, velocidade, direção)
data = data.iloc[:, :3]

# Definindo a variável tempo
data['DataHora'] = pd.to_datetime(data.iloc[:, 0], format='%Y/%m/%d %H:%M:%S')
data['Data'] = data['DataHora'].dt.date
data['Hora'] = data['DataHora'].dt.time

# Reorganizar as colunas: Colocar as duas últimas colunas na frente
data = data[['Data', 'Hora', 'Velocidade', 'Direcao']]

# Manipulando velocidade
vel = data['Velocidade'] / 100  # a divisão por 100 transforma cm/s para m/s

# Manipulando direção
dir = data['Direcao']

# Decomposição vetorial
dm = 0  # Declinação magnética do local (ajuste conforme necessário)
alfa = 0  # Grau de rotação do canal (ajuste conforme necessário)
teta = 90 - (dir + dm) + alfa  # Definindo o ângulo teta para componente u
m = np.radians(teta)  # Transformando graus para radianos
vel_u = vel * np.cos(m)  # Decomposição do vetor
vel_v = vel * np.sin(m)  # O v pode servir para verificar a advecção

data['Componente_u'] = vel_u
data['Componente_v'] = vel_v

# Plotagem de Gráficos
plt.figure(figsize=(12, 8))

# Configuração do primeiro subplot
plt.subplot(2, 1, 1)
plt.plot(data['DataHora'], vel_u, 'k', linewidth=1.1)
plt.ylabel('Velocidade (m/s)', fontname='Times New Roman', fontsize=14)
plt.ylim([-5, 70])
plt.title('Velocidade x Direção da Corrente P1', fontname='Times New Roman', fontsize=18)
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%d/%m/%y %H:%M'))
plt.gca().xaxis.set_major_locator(mdates.MinuteLocator(interval=10))
plt.gcf().autofmt_xdate()
plt.grid(True, which='both')

# Configuração do segundo subplot
plt.subplot(2, 1, 2)
plt.plot(data['DataHora'], dir, 'k*', markersize=4)
plt.ylabel('Direção (graus)', fontname='Times New Roman', fontsize=18)
plt.ylim([-1, 361])
plt.xlabel('Tempo (dia/mês/ano hora:minuto)', fontname='Times New Roman', fontsize=18)
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%d/%m/%y %H:%M'))
plt.gca().xaxis.set_major_locator(mdates.MinuteLocator(interval=10))
plt.gcf().autofmt_xdate()
plt.grid(True, which='both')

# Salva a figura como um arquivo PNG
plt.savefig('vel_dir.png', format='png')
plt.show()

# FIM DO SCRIPT
